# MR efficacy testing and calibration

### Plan
1) create a large set of binary fatty acid/wax combinations

2) test all of the combinations on MR

3) test a few select combinations on the DSC

4) plot the values of the MR data compared to DSC date (pull correlation - should be x=y)


### Imports

In [73]:
import numpy as np
import gc
import pandas as pd

In [4]:
# opentrons
import opentrons
from opentrons import labware, instruments, robot

### Defining Pure Component

In [64]:
class Components:
    def __init__(self, name, molecular_weight = []):
        self.name = name
        self.molecular_weight = molecular_weight

In [86]:
oleic_acid = Components("oleic_acid", 282.47)
ethanol = Components("ethanol", 46.07)
dodecanoic_acid = Components("dodecanoic_acid", 200.32)
octanoic_acid = Components("octanoic_acid", 144.21)
geranic_acid = Components("geranic_acid", 168.23)
candelilla_wax = Components("candelilla_wax")
light_mineral_oil = Components("light_mineral_oil", 425.363)
heavy_mineral_oil = Components("heavy_mineral_oil", 452.363)

In [87]:
for obj in gc.get_objects():
    if isinstance(obj, Components):
        print(obj.name)

oleic_acid
ethanol
dodecanoic_acid
octanoic_acid
geranic_acid
candelilla_wax
light_mineral_oil
heavy_mineral_oil


### Defining Stocks

In [44]:
class Stocks:
   def  __init__(self, molarity, composition, density = 1, aspiration_rate = 1):
        self.aspiration_rate = aspiration_rate # between 0 and 1 - multiplies default speed
        self.molarity = molarity #mol/L
        self.composition = composition
        self.density = density #g/ml

In [50]:
oleic_acid_2M = Stocks(2, ["oleic_acid", "ethanol"], .823, aspiration_rate = .8,)
dodecanoic_acid_1_3M = Stocks(1.3, ["dodecanoic_acid", "ethanol"], .803, aspiration_rate =.4)
octanoic_acid_2M = Stocks(2, ["octanoic_acid", "ethanol"], .820, aspiration_rate =.4)

In [62]:
for obj in gc.get_objects():
    if isinstance(obj, Stocks):
        print("Stock made of", obj.composition, "at", obj.molarity, "Molarity")

Stock made of ['oleic_acid', 'ethanol'] at 2 Molarity
Stock made of ['dodecanoic_acid', 'ethanol'] at 1.3 Molarity
Stock made of ['octanoic_acid', 'ethanol'] at 2 Molarity


### Pipetting Scheme
Each stock will ratio (volume) from 1:0 to 0:1 with every other stock - should result in 11 samples for each combo. There should be 21 combo's total. So 231 samples total creating 2.5 plates (3 plates of samples.)

##### Metrics
1) Total Samples = 231

2) Total Volume = 1000ul -or- 1 mL

In [83]:
comp1 = [1,.9,.8,.7,.6,.5,.4,.3,.2,.1,0]
comp2 = [0,.1,.2,.3,.4,.5,.6,.7,.8,.9,1]
df = pd.DataFrame(list(zip(comp1, comp2)), columns = ['comp1','comp2'])
df

,comp1,comp2
0,1.0,0.0
1,0.9,0.1
2,0.8,0.2
3,0.7,0.3
4,0.6,0.4
5,0.5,0.5
6,0.4,0.6
7,0.3,0.7
8,0.2,0.8
9,0.1,0.9


In [84]:
volume = 1000 #ul